In [1]:
import mysql.connector
import json, os, re
from datetime import datetime
import pandas as pd
import re
base_dir = os.path.join(os.path.dirname(os.getcwd())+"\\sql_learn\\Feb-25")
from query import *

def get_json_paths(root_folder):
    json_paths = []
    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name)
        if os.path.isfile(folder_path) and folder_path.endswith(".json"):
            json_paths.append(folder_path)
    return json_paths

def load_all_records(json_paths):
    all_records = {}
    for path in json_paths:
        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)
                records = data.get("records", [])
                name = " ".join(records[0].get("value", {}).get("amc_name", "").split(" ")[:2])
                if name not in all_records:
                    all_records[name] = records
                else:
                    name += " 2"
                    all_records[name] = records
        except Exception as e:
            print(f"Error reading {path}: {e}")
    return all_records

def load_all_records_mydata(json_paths):
    all_records = {}
    for path in json_paths:
        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)  # The whole dict is the data
                sample_scheme = next(iter(data.values()))
                amc_name = sample_scheme.get("amc_name", "").strip()
                name = " ".join(amc_name.split(" ")[:2])
                
                if name not in all_records:
                    all_records[name] = data
                else:
                    name += " 2"
                    all_records[name] = data
        except Exception as e:
            print(f"Error reading {path}: {e}")
    return all_records


In [ ]:
#mydatainsert
base_dir = os.path.join(os.path.dirname(os.getcwd())+"\\sql_learn\\json\\FEB25DATA") #FEB25DATA
json_paths = get_json_paths(base_dir)
combined_records = load_all_records_mydata(json_paths)

conn = mysql.connector.connect(
    host="172.22.225.155",
    user="cog_mf",
    password="bnYwFChjLAV2Z%9E",
    database="cog_mf",
    port=3306,
    charset='utf8mb4'
)
cursor = conn.cursor()

def null_if_empty(value):
    if isinstance(value, str) and value.strip() == "":
        return None
    return value


def insert_common_fund_data(cursor, details, curr_time, amc_month, data_from):
    keys = [
        "entered_time", "amc_for_month", "data_from", "amc_name", "main_scheme_name", "fund_name", #mutual_fund_name
        "benchmark_index", "monthly_aaum_date", "monthly_aaum_value", "scheme_launch_date",
        "min_addl_amt", "min_addl_amt_multiple", "min_amt", "min_amt_multiple",
        "entry_load", "exit_load",
        "alpha", "arithmetic_mean_ratio", "average_div_yld", "average_pb", "average_pe", "avg_maturity",
        "beta", "correlation_ratio", "downside_deviation", "information_ratio", "macaulay",
        "mod_duration", "port_turnover_ratio", "r_squared_ratio", "roe_ratio", "sharpe", "sortino_ratio",
        "std_dev", "tracking_error", "treynor_ratio", "upside_deviation", "ytm"
    ]
    
    metrics = details.get("metrics", {})
    load = details.get("load", {})
    
    values = [
        null_if_empty(curr_time),
        null_if_empty(amc_month),
        null_if_empty(data_from),
        null_if_empty(details.get("amc_name", "")),
        null_if_empty(details.get("main_scheme_name", "")),
        null_if_empty(details.get("mutual_fund_name", "")),
        null_if_empty(details.get("benchmark_index", "")),
        null_if_empty("20250228|February 2025"),
        null_if_empty(details.get("monthly_aaum_value", "")),  # Or make this dynamic again
        null_if_empty(details.get("scheme_launch_date", "")),
        null_if_empty(details.get("min_addl_amt", "")),
        null_if_empty(details.get("min_addl_amt_multiple", "")),
        null_if_empty(details.get("min_amt", "")),
        null_if_empty(details.get("min_amt_multiple", "")),
        null_if_empty(load.get("entry_load", "")),
        null_if_empty(load.get("exit_load", "")),
    ] + [null_if_empty(metrics.get(k, "")) for k in keys[16:]]

    query = f"""
        INSERT INTO mf_common_fund_data ({', '.join(keys)})
        VALUES ({', '.join(['%s'] * len(keys))})
    """

    try:
        cursor.execute(query, values)
        return cursor.lastrowid
    except Exception as e:
        print("Error inserting common fund data:")
        print("Scheme Name:", details.get("main_scheme_name"))
        print("Exception:", e)
        return None

def insert_fund_manager_data(cursor, mutual_fund_id, details, curr_time, amc_month, data_from):
    if not isinstance(details.get("fund_manager"), list):
        return
    for manager in details["fund_manager"]:
        query = """
            INSERT INTO mf_common_fund_manager_data
            (mutual_fund_id, entered_time, amc_for_month, data_from, amc_name,
             mutual_fund_name, main_scheme_name, name, qualification, managing_fund_since, total_exp)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        values = [
            mutual_fund_id,
            null_if_empty(curr_time),
            null_if_empty(amc_month),
            null_if_empty(data_from),
            null_if_empty(details.get("amc_name", "")),
            null_if_empty(details.get("mutual_fund_name", "")),
            null_if_empty(details.get("main_scheme_name", "")),
            null_if_empty(manager.get("name", "")),
            null_if_empty(manager.get("qualification", "")),
            null_if_empty(manager.get("managing_fund_since", "")),
            null_if_empty(manager.get("total_exp", ""))
        ]
        try:
            cursor.execute(query, values)
        except Exception as e:
            print(f"Error inserting fund manager: {e}")

# Usage
curr_time = datetime.now().strftime("%Y-%m-%d %H:%M")
amc_month = "FEB25"
data_from = "mydata"

for amc_name, schemes in combined_records.items():
    print(f"Doing: {amc_name}")
    for scheme_name, details in schemes.items():
        try:
            mutual_fund_id = insert_common_fund_data(cursor, details, curr_time, amc_month, data_from)
            if mutual_fund_id:
                insert_fund_manager_data(cursor, mutual_fund_id, details, curr_time, amc_month, data_from)
            else:
                print(f"Skipped Data Insertion for Scheme {scheme_name}")
        except Exception as e:
            print(f"Error inserting scheme '{scheme_name}' under AMC '{amc_name}': {e}")
    print(f"Done {amc_name}")
conn.commit()
cursor.close()
conn.close()
print("Working!!")


In [ ]:
#finkinsert
base_dir = os.path.join(os.path.dirname(os.getcwd())+"\\sql_learn\\json\\Feb-25")
json_paths = get_json_paths(base_dir)
combined_records = load_all_records(json_paths)

conn = mysql.connector.connect(
    host="172.22.225.155",
    user="cog_mf",
    password="bnYwFChjLAV2Z%9E",
    database="cog_mf",
    port=3306,
    charset='utf8mb4'
)

cursor = conn.cursor()

def null_if_empty(value):
    if isinstance(value, str) and value.strip() == "":
        return None
    return value

def insert_common_fund_data(cursor, details, curr_time, amc_month, data_from):
    keys = [
        "entered_time", "amc_for_month", "data_from", "amc_name", "main_scheme_name", "fund_name",
        "benchmark_index", "monthly_aaum_date", "monthly_aaum_value", "scheme_launch_date",
        "min_addl_amt", "min_addl_amt_multiple", "min_amt", "min_amt_multiple",
        "entry_load", "exit_load",
        "alpha", "arithmetic_mean_ratio", "average_div_yld", "average_pb", "average_pe", "avg_maturity",
        "beta", "correlation_ratio", "downside_deviation", "information_ratio", "macaulay",
        "mod_duration", "port_turnover_ratio", "r_squared_ratio", "roe_ratio", "sharpe", "sortino_ratio",
        "std_dev", "tracking_error", "treynor_ratio", "upside_deviation", "ytm"
    ]

    metrics = details.get("metrics", {})
    load = details.get("load", {})

    values = [
        null_if_empty(curr_time),
        null_if_empty(amc_month),
        null_if_empty(data_from),
        null_if_empty(details.get("amc_name", "")),
        null_if_empty(details.get("main_scheme_name", "")),
        null_if_empty(details.get("mutual_fund_name", "")),
        null_if_empty(", ".join(details.get("benchmark_index", [])) if isinstance(details.get("benchmark_index"), list) else details.get("benchmark_index", "")),
        null_if_empty(details.get("monthly_aaum_date", "")),
        null_if_empty(details.get("monthly_aaum_value", "")),
        null_if_empty(details.get("scheme_launch_date", "")),
        null_if_empty(details.get("min_addl_amt", "")),
        null_if_empty(details.get("min_addl_amt_multiple", "")),
        null_if_empty(details.get("min_amt", "")),
        null_if_empty(details.get("min_amt_multiple", "")),
        null_if_empty(next((l.get("comment") for l in details.get("load", []) if l.get("type") == "entry"), "")),
        null_if_empty(next((l.get("comment") for l in details.get("load", []) if l.get("type") == "exit"), ""))
    ] + [null_if_empty(next((m.get("value") for m in details.get("metrics", []) if m.get("name") == key), "")) for key in keys[16:]]

    query = f"""
        INSERT INTO mf_common_fund_data ({', '.join(keys)})
        VALUES ({', '.join(['%s'] * len(keys))})
    """

    try:
        cursor.execute(query, values)
        return cursor.lastrowid
    except Exception as e:
        print("Error inserting common fund data:")
        print("Scheme Name:", details.get("main_scheme_name"))
        print("Exception:", e)
        return None

def insert_fund_manager_data(cursor, mutual_fund_id, details, curr_time, amc_month, data_from):
    if not isinstance(details.get("fund_manager"), list):
        return
    for manager in details["fund_manager"]:
        query = """
            INSERT INTO mf_common_fund_manager_data
            (mutual_fund_id, entered_time, amc_for_month, data_from, amc_name,
             mutual_fund_name, main_scheme_name, name, qualification, managing_fund_since, total_exp)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        values = [
            mutual_fund_id,
            null_if_empty(curr_time),
            null_if_empty(amc_month),
            null_if_empty(data_from),
            null_if_empty(details.get("amc_name", "")),
            null_if_empty(details.get("mutual_fund_name", "")),
            null_if_empty(details.get("main_scheme_name", "")),
            null_if_empty(manager.get("name", "")),
            null_if_empty(manager.get("qualification", "")),
            null_if_empty(manager.get("managing_fund_since", "")),
            null_if_empty(manager.get("total_exp", ""))
        ]
        try:
            cursor.execute(query, values)
        except Exception as e:
            print(f"Error inserting fund manager: {e}")

# Start Insertion
curr_time = datetime.now().strftime("%Y-%m-%d %H:%M")
amc_month = "FEB25"
data_from = "finkstein"

for amc_name, records in combined_records.items():
    print(f"Inserting AMC: {amc_name}")
    for record in records:
        try:
            details = record["value"] 
            mutual_fund_id = insert_common_fund_data(cursor, details, curr_time, amc_month, data_from)
            if mutual_fund_id:
                insert_fund_manager_data(cursor, mutual_fund_id, details, curr_time, amc_month, data_from)
            else:
                print(f"Skipped: {details.get('main_scheme_name')}")
        except Exception as e:
            print(f"Error processing scheme under AMC '{amc_name}': {e}")


conn.commit()
cursor.close()
conn.close()
print("Finkstein data insertion completed.")
